In [1]:
#여기 코드가 가장 중요하다. 위의 코드를 사용하면 matplot창을 띄워주고
# 아래의 코드를 사용하면 파이썬에서 첫번째 이미지만 보여준다.


#%matplotlib qt

%matplotlib inline

In [2]:
import tensorflow as tf
import os

def download_model(model_type):
    server_prefix = 'https://omnomnom.vision.rwth-aachen.de/data/metrabs'
    model_zippath = tf.keras.utils.get_file(
        origin=f'{server_prefix}/{model_type}_20211019.zip',
        extract=True, cache_subdir='models')
    model_path = os.path.join(os.path.dirname(model_zippath), model_type)
    return model_path

model = tf.saved_model.load(download_model('metrabs_mob3l_y4t')) # or metrabs_eff2l_y4 for the big model
! wget -q https://raw.githubusercontent.com/isarandi/metrabs/master/img/test_image_3dpw.jpg


'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


## area2 이미지 3장 분석


In [ ]:
import cv2
image1 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/area2/area2_244.jpg')
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
pred1 = model.detect_poses(image1, skeleton='smpl_24')

In [110]:
import cv2
image2 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/area2/area2_145.jpg')
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
pred2 = model.detect_poses(image2, skeleton='smpl_24')

In [113]:
import cv2
image3 = cv2.imread('C:/Users/Leeyourack/jookyoung/image/area2/area2_230.jpg')
image3 = cv2.cvtColor(image3, cv2.COLOR_BGR2RGB)
pred3 = model.detect_poses(image3, skeleton='smpl_24')

### area2_normal vector

In [115]:
poses3d1=pred1['poses3d'].numpy()[0]
poses3d1[..., 1], poses3d1[..., 2] = poses3d1[..., 2], -poses3d1[..., 1]

In [116]:
a=poses3d1[10]
a

array([ 760.45, 4092.  , -492.7 ], dtype=float32)

In [117]:
poses3d1.T[0]

array([439.6   , 441.4   , 548.    , 378.35  , 472.35  , 779.5   ,
       278.8   , 723.2   , 847.4   , 254.775 , 760.45  , 926.3   ,
       115.0375, 107.6375, 240.375 ,  80.825 ,  28.475 , 314.55  ,
       101.95  , 528.1   , 191.875 , 599.9   , 223.    , 653.9   ],
      dtype=float32)

In [118]:
poses3d2=pred2['poses3d'].numpy()[0]
poses3d2[..., 1], poses3d2[..., 2] = poses3d2[..., 2], -poses3d2[..., 1]

In [119]:
b=poses3d2[10]
b

array([ 151.425, 6094.4  ,  359.45 ], dtype=float32)

In [120]:
poses3d3=pred3['poses3d'].numpy()[0]
poses3d3[..., 1], poses3d3[..., 2] = poses3d3[..., 2], -poses3d3[..., 1]
 

In [121]:
c=poses3d3[10]
c

array([  567.3,  4154.8, -1113.2], dtype=float32)

In [122]:
ab = b-a
print(ab)
ac = c-a
print(ac)

[-609.025  2002.3999  852.15  ]
[-193.15002    62.799805 -620.49994 ]


In [123]:
list = [ab[1]*ac[2]-ab[2]*ac[1],-(ab[0]*ac[2]-ab[2]*ac[0]),ab[0]*ac[1]-ab[1]*ac[0]]
list

[-1296003.9, -542492.8, 348516.94]

In [124]:
list = [(ab[1]*ac[2]-ab[2]*ac[1])/100000,
        -(ab[0]*ac[2]-ab[2]*ac[0])/100000,
            (ab[0]*ac[1]-ab[1]*ac[0])/100000]
list

[-12.96003875, -5.424928125, 3.485169375]

In [17]:
list[0]

-2.34740375

### rotation matrix

In [125]:
x=list[0]
y=list[1]
z=list[2]
print('x:',x, 'y:',y, 'z:',z )
xy = (x**2+y**2)**0.5
yz = (y**2+z**2)**0.5
xz = (x**2+z**2)**0.5
print('xy:',xy, 'yz:',yz, 'xz:',xz )

xyz = (x**2+y**2+z**2)**0.5
print('xyz:',xyz)


x: -12.96003875 y: -5.424928125 z: 3.485169375
xy: 14.049642328647288 yz: 6.447964852095109 xz: 13.420469811967816
xyz: 14.475456992280261


In [126]:
#z축 회전

import math
cos1 = math.cos(math.pi/2+abs(x/xy))
sin1 = math.sin(math.pi/2+abs(x/xy))


print(cos1,sin1)

-0.7970811770576316 0.6038721695693722


In [20]:
math.cos(math.pi/2)

6.123233995736766e-17

In [127]:
#y축 회전
import math
cos2 = math.cos(math.pi/2-abs(xy/xyz))
sin2 = math.sin(math.pi/2-abs(xy/xyz))
sin2 = -sin2

print(cos2,sin2)

0.825215527282814 -0.564817964950963


In [22]:
import math
math.cos(math.pi*1/2)

6.123233995736766e-17

In [23]:
import numpy as np

r1 = np.array([[cos1,-sin1,0], #z축 회전
                [sin1,cos1,0],
                [0,0,1]])
r2 = np.array([[cos2,0,sin2],
                [0,1,0],
                [-sin2,0,cos2]])        #y축 회전
r3 = np.array([[0,1,0], #z축 회전
                [-1,0,0],
                [0,0,1]])
                

In [128]:
import numpy as np

r1 = np.array([[cos1,-sin1,0],          #z축 회전
                [sin1,cos1,0],
                [0,0,1]])
r2 = np.array([[cos2,0,sin2],           #y축 회전
                [0,1,0],
                [-sin2,0,cos2]])        
r3 = np.array([[math.cos(math.pi*1/1.6),math.sin(math.pi*1/1.6),0],                 #z축 회전
                [-math.sin(math.pi*1/1.6),math.cos(math.pi*1/1.6),0],
                [0,0,1]])

In [25]:
r3

array([[-0.38268343,  0.92387953,  0.        ],
       [-0.92387953, -0.38268343,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [129]:
rotation1 = r2@r1
rotation1

array([[-0.65776376, -0.49832469, -0.56481796],
       [ 0.60387217, -0.79708118,  0.        ],
       [-0.45020577, -0.34107785,  0.82521553]])

## 변환행렬 적용 전

In [130]:
# 변환적용 전

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image1)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    '''a=np.ones(24)*(-6000)
    b=np.ones(24)
    c=np.ones(24)*(-2700)
    shift=np.row_stack((a,b,c))'''
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        #p1=pose3d.T+shift
        #pose3d=rotation1@pose3d.T
        #pose3d=pose3d+shift
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d[i_start], pose3d[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d.T, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image1, 
    pred1['boxes'].numpy(),
    pred1['poses3d'].numpy(),
    pred1['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

## 변환행렬 적용 후

In [132]:
##1
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image1)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(0)
    b=np.ones(24)*(+19000)
    c=np.ones(24)*(+2300)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift

        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image1, 
    pred1['boxes'].numpy(),
    pred1['poses3d'].numpy(),
    pred1['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

In [76]:
##2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image2)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(-11000)
    b=np.ones(24)*(+4000)
    c=np.ones(24)*(+2500)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift

        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image2, 
    pred2['boxes'].numpy(),
    pred2['poses3d'].numpy(),
    pred2['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())

In [30]:
##3
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
import numpy as np 

def visualize(im, detections, poses3d, poses2d, edges):
    fig = plt.figure(figsize=(10, 5.2))
    image_ax = fig.add_subplot(1, 2, 1)
    image_ax.imshow(image3)
    for x, y, w, h in detections[:, :4]:
        image_ax.add_patch(Rectangle((x, y), w, h, fill=False))
    

    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(14, -76)
    pose_ax.set_xlim3d(-10000, 1500)
    pose_ax.set_zlim3d(0, 6000)
    pose_ax.set_ylim3d(0, 20000)


    # x, y, z값 
    xpos = [-10000,-1500,500]
    ypos = [10000,9000,5000]
    zpos = [0,0,0]

    # delta x, y, z 값
    dx = [4000,2000,1000]
    dy = [10000,11000,15000]
    dz = [2000,2000,2000]

    #그래프 만들기
    pose_ax.bar3d(xpos,ypos,zpos,dx,dy,dz,'#7f7f7f',alpha=0.2) #,color='#00ceaa'(초록색) 기본값은 파란색
    pose_ax.set_title('6F')
    pose_ax.set_xlabel('x axis')
    pose_ax.set_ylabel('y axis')
    pose_ax.set_zlabel('z axis')

    '''#2
    pose_ax = fig.add_subplot(1, 2, 2, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-500, 1500)
    pose_ax.set_ylim3d(0, 3000)'''



    # Matplotlib plots the Z axis as vertical, but our poses have Y as the vertical axis.
    # Therefore, we do a 90° rotation around the X axis:
    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1] 
    a=np.ones(24)*(-11000)
    b=np.ones(24)*(+4000)
    c=np.ones(24)*(+2500)
    shift=np.row_stack((a,b,c))
    

    for pose3d, pose2d in zip(poses3d, poses2d):
        
        pose3d=rotation1@pose3d.T
        pose3d=pose3d+shift
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
            pose_ax.plot(*zip(pose3d.T[i_start], pose3d.T[i_end]), marker='o', markersize=2)
        image_ax.scatter(*pose2d.T, s=2)
        pose_ax.scatter(*pose3d, s=2)

    fig.tight_layout()
    plt.show()

visualize(
    image3, 
    pred3['boxes'].numpy(),
    pred3['poses3d'].numpy(),
    pred3['poses2d'].numpy(),
    model.per_skeleton_joint_edges['smpl_24'].numpy())